<a href="https://colab.research.google.com/github/nithishsriramk98/Python-scripts/blob/main/defects_in_materials.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ase pymatgen shakenbreak doped

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 74.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.9/205.9 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 398.4/398.4 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.4/277.4 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.3/89.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.3/332.3 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.2/198.2 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [ ]:
from google.colab import files

uploaded = files.upload()
# It will ask you to choose your QE input file (e.g., scf.in)
qe_input_file = list(uploaded.keys())[0]
qe_input_file


Saving scf.in to scf (2).in


'scf (2).in'

In [ ]:
with open ('scf (2).in', 'r') as f:
  print(f.read())

&CONTROL
    calculation     = 'scf',
    prefix          = 'nithish',
    verbosity       = 'high'
    outdir          = '/scratch/vinoth.mku/temp/',
    pseudo_dir      = './',
    forc_conv_thr   = 1.d-5,
    etot_conv_thr   = 1.d-8,
/

&SYSTEM
    ibrav = 0
    nat   = 6
    ntyp  = 2
    ecutwfc = 90
    nbnd = 100
/

&ELECTRONS
conv_thr = 1.D-9
/

CELL_PARAMETERS angstrom
  4.2746	0.0000		0.0000
  0.0000	4.2746		0.0000
  0.0000	0.0000		4.2746

ATOMIC_SPECIES
Cu    63.54600  Cu.upf
O     15.99940  O.upf

ATOMIC_POSITIONS (angstrom)
Cu            1.0686530272        1.0686530272        3.2059580751
Cu            1.0686530272        3.2059580751        1.0686530272
Cu            3.2059580751        3.2059580751        3.2059580751
Cu            3.2059580751        1.0686530272        1.0686530272
O             0.0000000000       -0.0000000000        0.0000000000
O             2.1373060544        2.1373060544        2.1373060544

K_POINTS {automatic}
 5 5 5 0 0 0
 
 HUBBARD ortho-ato

In [ ]:
from ase.io import read, write
from ase.build import make_supercell
import numpy as np

input_file = qe_input_file
supercell_cif = "supercell_2x2x2.cif"
supercell_qe = "supercell_2x2x2.in"

atoms = read(input_file, format="espresso-in")

T = np.diag([2, 2, 2])
supercell = make_supercell(atoms, T)
supercell.wrap()

write(supercell_cif, supercell)

# ---- FIX: PROVIDE PSEUDOPOTENTIAL MAP ----
pseudos = {
    "Cu": "Cu.UPF",   # change to YOUR actual filename from ATOMIC_SPECIES
    "O" : "O.UPF",
    "Sr": "Sr.UPF",
    "Bi": "Bi.UPF",
    "Br": "Br.UPF"
}

write(supercell_qe, supercell, format="espresso-in",
      pseudopotentials=pseudos)

print("Supercell QE input written correctly!")


Supercell QE input written correctly!


In [ ]:
from google.colab import files

files.download("supercell_2x2x2.cif")
files.download("supercell_2x2x2.in")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install ase pymatgen doped

In [ ]:
pseudo_map = {
    "Cu": "Cu.pbe-dn-kjpaw_psl.1.0.0.UPF",
    "O": "O.pbe-n-kjpaw_psl.1.0.0.UPF",
}

In [ ]:
from ase.io import read
from pymatgen.io.ase import AseAtomsAdaptor
from doped.generation import DefectsGenerator

qe_supercell_file = "supercell_2x2x2.in"

print("Reading QE supercell...")
ase_struct = read(qe_supercell_file, format="espresso-in")
pmg_struct = AseAtomsAdaptor.get_structure(ase_struct)

print(f"Atoms: {len(ase_struct)}")
print(f"Species: {set(ase_struct.get_chemical_symbols())}")

print("\nGenerating intrinsic defects...")
gen = DefectsGenerator(pmg_struct)

# THIS contains all defects:
defects = gen.defect_entries

print(f"Total defects generated: {len(defects)}")
for name in defects.keys():
    print("  ✔", name)


Reading QE supercell...
Atoms: 48
Species: {'Cu', 'O'}

Generating intrinsic defects...


Generating DefectEntry objects: 100.0%|██████████| [00:12,   7.73it/s]

Vacancies    Guessed Charges    Conv. Cell Coords    Wyckoff
-----------  -----------------  -------------------  ---------
v_Cu         [+1,0,-1]          [0.250,0.250,0.250]  4b
v_O          [+2,+1,0,-1]       [0.000,0.000,0.000]  2a

Substitutions    Guessed Charges    Conv. Cell Coords    Wyckoff
---------------  -----------------  -------------------  ---------
Cu_O             [+4,+3,+2,+1,0]    [0.000,0.000,0.000]  2a
O_Cu             [0,-1,-2,-3]       [0.250,0.250,0.250]  4b

Interstitials    Guessed Charges    Conv. Cell Coords    Wyckoff
---------------  -----------------  -------------------  ---------
Cu_i_D2d         [+2,+1,0]          [0.000,0.000,0.500]  6d
Cu_i_D3d         [+2,+1,0]          [0.750,0.750,0.750]  4c
O_i_D2d          [0,-1,-2]          [0.000,0.000,0.500]  6d
O_i_D3d          [0,-1,-2]          [0.750,0.750,0.750]  4c

The number in the Wyckoff label is the site multiplicity/degeneracy of that defect in the conventional ('conv.') unit cell, which compris

In [ ]:
from ase.io import write
from pymatgen.io.ase import AseAtomsAdaptor

# === USER SETTINGS: MODIFY HERE ===

pseudo_dir = "./pseudos"
pseudo_map = {
    "O": "O.pbe-n-rrkjus_psl.1.0.0.UPF",
    "Cu": "Cu.pbe-dn-rrkjus_psl.1.0.0.UPF"
}

control_block = """
&control
    calculation = 'scf',
    prefix = 'DEFECT',
    pseudo_dir = './pseudos',
    outdir = './tmp'
/
"""

system_block = """
&system
    ibrav = 0,
    ecutwfc = 60,
    ecutrho = 480,
    occupations = 'smearing',
    smearing = 'mp',
    degauss = 0.01
/
"""

electrons_block = """
&electrons
    diagonalization = 'david',
    conv_thr = 1.0d-8,
    mixing_beta = 0.3
/
"""

ions_block = """
&ions
/
"""

cell_block = """
&cell
/
"""

# === FUNCTION TO WRITE QE INPUT ===

def write_qe_input(struct, filename, prefix):
    """Writes a complete QE input file for a defect structure."""

    atoms = AseAtomsAdaptor.get_atoms(struct)
    species = set(atoms.get_chemical_symbols())

    # Pseudopotential section
    pp_lines = ""
    for sp in species:
        if sp not in pseudo_map:
            raise ValueError(f"No pseudopotential defined for element {sp}")
        pp_lines += f"{sp}  {pseudo_map[sp]}\n"

    # ATOMIC_SPECIES block
    atomic_species = "ATOMIC_SPECIES\n" + pp_lines

    # ATOMIC_POSITIONS
    atomic_positions = "ATOMIC_POSITIONS angstrom\n"
    for s, pos in zip(atoms.get_chemical_symbols(), atoms.positions):
        atomic_positions += f"{s}  {pos[0]:.6f} {pos[1]:.6f} {pos[2]:.6f}\n"

    # CELL_PARAMETERS
    cp = atoms.get_cell()
    cell_parameters = "CELL_PARAMETERS angstrom\n"
    for v in cp:
        cell_parameters += f"{v[0]:.6f} {v[1]:.6f} {v[2]:.6f}\n"

    # Combine all blocks
    text = (
        control_block.replace("DEFECT", prefix)
        + system_block
        + electrons_block
        + ions_block
        + cell_block
        + atomic_species
        + atomic_positions
        + cell_parameters
    )

    with open(filename, "w") as f:
        f.write(text)

    return filename


# === WRITE EVERY DEFECT STRUCTURE ===

def write_all_qe_defects(defects):
    written = []

    for defect_name, entry in defects.items():
        print(f"Processing: {defect_name}")

        # THIS IS THE CORRECT WAY:
        struct = entry.defect.get_supercell_structure()

        safe = defect_name.replace(" ", "_")
        outname = f"{safe}.in"

        write_qe_input(struct, outname, prefix=safe)
        written.append(outname)

        print(f"  → written: {outname}")

    return written


In [ ]:
written_files = write_all_qe_defects(defects)
print("Done!")


Processing: v_Cu_+1
  → written: v_Cu_+1.in
Processing: v_Cu_0
  → written: v_Cu_0.in
Processing: v_Cu_-1
  → written: v_Cu_-1.in
Processing: v_O_+2
  → written: v_O_+2.in
Processing: v_O_+1
  → written: v_O_+1.in
Processing: v_O_0
  → written: v_O_0.in
Processing: v_O_-1
  → written: v_O_-1.in
Processing: Cu_O_+4
  → written: Cu_O_+4.in
Processing: Cu_O_+3
  → written: Cu_O_+3.in
Processing: Cu_O_+2
  → written: Cu_O_+2.in
Processing: Cu_O_+1
  → written: Cu_O_+1.in
Processing: Cu_O_0
  → written: Cu_O_0.in
Processing: O_Cu_0
  → written: O_Cu_0.in
Processing: O_Cu_-1
  → written: O_Cu_-1.in
Processing: O_Cu_-2
  → written: O_Cu_-2.in
Processing: O_Cu_-3
  → written: O_Cu_-3.in
Processing: Cu_i_D2d_+2
  → written: Cu_i_D2d_+2.in
Processing: Cu_i_D2d_+1
  → written: Cu_i_D2d_+1.in
Processing: Cu_i_D2d_0
  → written: Cu_i_D2d_0.in
Processing: Cu_i_D3d_+2
  → written: Cu_i_D3d_+2.in
Processing: Cu_i_D3d_+1
  → written: Cu_i_D3d_+1.in
Processing: Cu_i_D3d_0
  → written: Cu_i_D3d_0.in
Proc

In [ ]:
import zipfile
from google.colab import files

zip_name = "QE_defect_inputs.zip"

with zipfile.ZipFile(zip_name, 'w') as z:
    for f in written_files:       # written_files = list of all .in files generated earlier
        z.write(f)

files.download(zip_name)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>